In [3]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
import time
from numpy import concatenate
import numpy as np
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from tensorflow.python.keras.layers import Dense, LSTM , Dropout
from tensorflow.python.keras import Sequential
from math import sqrt;
import tensorflow as tf

In [4]:
url = "https://drive.google.com/file/d/1u95FUEFI29NV-LmdCRHzRwj-t4tWH87m/view?usp=share_link"
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [5]:
df.shape

(35064, 29)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil coal-derived gas           35046 non-null  float64
 4   generation fossil gas                        35046 non-null  float64
 5   generation fossil hard coal                  35046 non-null  float64
 6   generation fossil oil                        35045 non-null  float64
 7   generation fossil oil shale                  35046 non-null  float64
 8   generation fossil peat                       35046 non-null  float64
 9   generation geothermal                        35046 non-null  float64
 10

In [7]:
df.describe()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
count,35045.000000,35046.000000,35046.0,35046.000000,35046.000000,35045.000000,35046.0,35046.0,35046.0,0.0,...,35045.000000,35046.0,35046.000000,35064.000000,0.0,35064.000000,35064.000000,35028.000000,35064.000000,35064.000000
mean,383.513540,448.059208,0.0,5622.737488,4256.065742,298.319789,0.0,0.0,0.0,NaN,...,269.452133,0.0,5464.479769,1439.066735,NaN,5471.216689,28712.129962,28696.939905,49.874341,57.884023
std,85.353943,354.568590,0.0,2201.830478,1961.601013,52.520673,0.0,0.0,0.0,NaN,...,50.195536,0.0,3213.691587,1677.703355,NaN,3176.312853,4594.100854,4574.987950,14.618900,14.204083
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,...,0.000000,0.0,0.000000,0.000000,NaN,237.000000,18105.000000,18041.000000,2.060000,9.330000
25%,333.000000,0.000000,0.0,4126.000000,2527.000000,263.000000,0.0,0.0,0.0,NaN,...,240.000000,0.0,2933.000000,69.000000,NaN,2979.000000,24793.750000,24807.750000,41.490000,49.347500
50%,367.000000,509.000000,0.0,4969.000000,4474.000000,300.000000,0.0,0.0,0.0,NaN,...,279.000000,0.0,4849.000000,576.000000,NaN,4855.000000,28906.000000,28901.000000,50.520000,58.020000
75%,433.000000,757.000000,0.0,6429.000000,5838.750000,330.000000,0.0,0.0,0.0,NaN,...,310.000000,0.0,7398.000000,2636.000000,NaN,7353.000000,32263.250000,32192.000000,60.530000,68.010000
max,592.000000,999.000000,0.0,20034.000000,8359.000000,449.000000,0.0,0.0,0.0,NaN,...,357.000000,0.0,17436.000000,5836.000000,NaN,17430.000000,41390.000000,41015.000000,101.990000,116.800000


In [8]:
df.isnull().sum()

time                                               0
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar                              

In [9]:
df = df.drop(['time', 'generation biomass', 
'generation fossil brown coal/lignite', 'generation fossil coal-derived gas', 'generation fossil hard coal','generation fossil oil shale',
'generation fossil peat',                            
'generation geothermal' ,                            
'generation hydro pumped storage aggregated',     
'generation hydro pumped storage consumption',       
'generation hydro run-of-river and poundage',                         
'generation marine',                                 
'generation nuclear',                                
'generation other',                                  
'generation other renewable',                        
'generation solar',                                  
'generation waste',                                  
'generation wind offshore',                         
'generation wind onshore',                                                     
'forecast wind onshore day ahead', 
'forecast wind offshore eday ahead',                  
'total load forecast',                                                               
'price day ahead',                                   
                                      ], axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   generation fossil gas             35046 non-null  float64
 1   generation fossil oil             35045 non-null  float64
 2   generation hydro water reservoir  35046 non-null  float64
 3   forecast solar day ahead          35064 non-null  float64
 4   total load actual                 35028 non-null  float64
 5   price actual                      35064 non-null  float64
dtypes: float64(6)
memory usage: 1.6 MB


In [11]:
# for i in df.columns:
#   if df[i].isnull().sum() > 0:
#     df[i] = df[i].fillna(df[i].mean())

In [12]:
df.isnull().sum()

generation fossil gas               18
generation fossil oil               19
generation hydro water reservoir    18
forecast solar day ahead             0
total load actual                   36
price actual                         0
dtype: int64

In [13]:
df.corr()['total load actual']

generation fossil gas               0.548913
generation fossil oil               0.497089
generation hydro water reservoir    0.479481
forecast solar day ahead            0.404362
total load actual                   1.000000
price actual                        0.436127
Name: total load actual, dtype: float64

In [14]:
df.columns

Index(['generation fossil gas', 'generation fossil oil',
       'generation hydro water reservoir', 'forecast solar day ahead',
       'total load actual', 'price actual'],
      dtype='object')

In [15]:
values = df.values

In [16]:
values.shape

(35064, 6)

In [17]:
scaler = MinMaxScaler(feature_range=(0, 1))
# scaledMin = scaler.fit_transform(valuesMin)
scaled = scaler.fit_transform(values)

In [18]:
#covert to time-series
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [19]:
reframed = series_to_supervised(scaled, 1, 1)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t)
1,0.241789,0.360802,0.195210,0.002913,0.319666,0.521820,0.259359,0.351893,0.170436,0.002742,0.276008,0.517261
2,0.259359,0.351893,0.170436,0.002742,0.276008,0.517261,0.242438,0.349666,0.140933,0.001371,0.204274,0.513166
3,0.242438,0.349666,0.140933,0.001371,0.204274,0.513166,0.215334,0.356347,0.080078,0.000343,0.141247,0.465153
4,0.215334,0.356347,0.080078,0.000343,0.141247,0.465153,0.206150,0.347439,0.074013,0.001542,0.096762,0.434633
5,0.206150,0.347439,0.074013,0.001542,0.096762,0.434633,0.201557,0.347439,0.076377,0.000685,0.081135,0.412208


In [20]:
reframed.shape

(34994, 12)

In [21]:
reframed.drop(reframed.columns[[-1]], axis=1, inplace=True)

In [22]:
reframed.shape

(34994, 11)

In [23]:
reframed.drop(reframed.columns[[x for x in range(7, 10)]], axis=1, inplace=True)

In [24]:
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var1(t),var5(t)
1,0.241789,0.360802,0.195210,0.002913,0.319666,0.521820,0.259359,0.276008
2,0.259359,0.351893,0.170436,0.002742,0.276008,0.517261,0.242438,0.204274
3,0.242438,0.349666,0.140933,0.001371,0.204274,0.513166,0.215334,0.141247
4,0.215334,0.356347,0.080078,0.000343,0.141247,0.465153,0.206150,0.096762
5,0.206150,0.347439,0.074013,0.001542,0.096762,0.434633,0.201557,0.081135


In [25]:
reframed.drop(reframed.columns[[-2]], axis=1, inplace=True)

In [26]:
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var5(t)
1,0.241789,0.360802,0.195210,0.002913,0.319666,0.521820,0.276008
2,0.259359,0.351893,0.170436,0.002742,0.276008,0.517261,0.204274
3,0.242438,0.349666,0.140933,0.001371,0.204274,0.513166,0.141247
4,0.215334,0.356347,0.080078,0.000343,0.141247,0.465153,0.096762
5,0.206150,0.347439,0.074013,0.001542,0.096762,0.434633,0.081135


In [27]:
reframed.columns

Index(['var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)',
       'var6(t-1)', 'var5(t)'],
      dtype='object')

In [28]:
#split into train and test sets
values = reframed.values
# values5 = reframed5.values
n_train = round(values.shape[0])

train = values[:, :]
test = values[:, :]

In [29]:
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [30]:
#reshap input menjadi 3D ([)samples, timesteps, features)
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [31]:
#design network LSTM
model = Sequential()
model.add(LSTM(32, return_sequences = True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units = 64, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 64, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 64))
model.add(Dropout(0.2))

model.add(Dense(1, activation='tanh'))

model.compile(loss='mse', optimizer='adam')

In [32]:
#fit network
start = time.time()
history = model.fit(train_X, train_y, epochs=500, batch_size=128, validation_data=(test_X, test_y), verbose=1, shuffle=False)
end = time.time()
print('Processing Time {} seconds.'.format(end-start))

Epoch 1/500


KeyboardInterrupt: ignored

In [ ]:
#plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.xlabel('Epoch')
pyplot.ylabel('Loss')
pyplot.legend()
pyplot.show()

In [ ]:
#membuat prediksi training
xhat = model.predict(train_X)
train_X = train_X.reshape((train_X.shape[0], train_X.shape[2]))

#invert scaling untuk peramalan
inv_xhat = concatenate((xhat, train_X[:, 1:]), axis=1)
inv_xhat = scaler.inverse_transform(inv_xhat)
inv_xhat = inv_xhat[:,0]

#invert scaling untuk aktual
train_y = train_y.reshape((len(train_y), 1))
inv_x = concatenate((train_y, train_X[:, 1:]), axis=1)
inv_x = scaler.inverse_transform(inv_x)
inv_x = inv_x[:,0]

#================================================================#

#membuat prediksi testing
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

#invert scaling untuk peramalan
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

#invert scaling untuk aktual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [ ]:
# calculate RMSE
#rmsetrain = np.sqrt(mean_squared_error(xhat, train_y))
#print(f'RMSE Training is : %.3f' % rmsetrain)
#rmsetrain = np.sqrt(mean_squared_error(inv_x, inv_xhat))
#print(f'RMSE Training is : %.3f' % rmsetrain)

rmsetest = np.sqrt(mean_squared_error(test_y, yhat))
print(f'RMSE Testing is : %.3f' % rmsetest)
#rmsetest = np.sqrt(mean_squared_error(inv_y, inv_yhat))
#print(f'RMSE Testing is : %.3f' % rmsetest)


def mean_absolute_percentage_error(yhat, test_y):
    yhat, test_y = np.array(yhat), np.array(test_y)
    return np.mean(np.abs((yhat, test_y) / test_y)) 

#print(f'MAPE train is : {(mean_absolute_percentage_error(train_y, xhat))}')
#print(f'MAPE train is : {(mean_absolute_percentage_error(inv_x, inv_xhat))}')

print(f'MAPE test is : {(mean_absolute_percentage_error(test_y, yhat))}')
#print(f'MAPE test is : {(mean_absolute_percentage_error(inv_y, inv_yhat))}')

#print(f'R2 train is : {r2_score(train_y, xhat)}')
#print(f'R2 train is : {r2_score(inv_x, inv_xhat)}')

print(f'R2 test is : {r2_score(test_y, yhat)}')
#print(f'R2 test is : {r2_score(inv_y, inv_yhat)}')


In [ ]:
print('Actual :', test_y)
print('Predicted:', yhat)
# plot history
pyplot.plot(test_y, label='Actual')
pyplot.plot(yhat, label='Forecasting')
pyplot.xlabel('Timestep')
pyplot.ylabel('Value')
pyplot.legend()
pyplot.show()

In [ ]:
idx = 200#int(len(yhat)*0.25)
aa=[x for x in range(idx)]
pyplot.figure(figsize=(20,4))
pyplot.plot(aa, test_y[:idx], marker='.', label="actual")
pyplot.plot(aa, yhat[:idx], 'r', label="prediction")
# plt.tick_params(left=False, labelleft=True) #remove ticks
pyplot.tight_layout()
sns.despine(top=True)
pyplot.subplots_adjust(left=0.07)
pyplot.ylabel('TOTAL Load', size=15)
pyplot.xlabel('Time step', size=15)
pyplot.legend(fontsize=15)
pyplot.show()